# Transfer Learning From Pre-Trained Model (VGG16)  for Face Recognition

### Loading the VGG16 Model

In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes but we are setting image size 100x100 to reduce computing power
img_rows = 224
img_cols = 224

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

### Inpsecting each layer

In [3]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


### Let's freeze all layers except the top 4 

In [4]:
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


### Let's make a function that returns our FC Head

In [5]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [6]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [7]:
model.layers

### Let's add our FC Head back onto VGG

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 5

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Loading our crap Dataset


!cp -r '/content/drive/My Drive/Tesis/Dataset-resized-224.rar (Unzipped Files)/trainingSet' '/content/trainingSet'
!cp -r '/content/drive/My Drive/Tesis/Dataset-resized-224.rar (Unzipped Files)/validationSet' '/content/validationSet'
!cp -r '/content/drive/My Drive/Tesis/Dataset-resized-224.rar (Unzipped Files)/testSet' '/content/testSet'

train_data_dir = '/content/trainingSet'
validation_data_dir = '/content/validationSet'
test_data_dir = '/content/testSet'

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/Agustin Pitaro/Documents/Tesis/Proyecto Final/Dataset/train'
validation_data_dir = 'C:/Users/Agustin Pitaro/Documents/Tesis/Proyecto Final/Dataset/validation'
test_data_dir = 'C:/Users/Agustin Pitaro/Documents/Tesis/Proyecto Final/Dataset/test'

In [10]:


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

 
# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_rows, img_cols),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 2800 images belonging to 5 classes.
Found 600 images belonging to 5 classes.
Found 600 images belonging to 5 classes.


### Training our top layers

In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("vgg16-transfer-learning-1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
#modelnew.compile(loss = 'categorical_crossentropy',
#              optimizer = RMSprop(lr = 0.001),
#              metrics = ['accuracy'])

nb_train_samples=train_generator.n//train_generator.batch_size
nb_validation_samples=validation_generator.n//validation_generator.batch_size
epochs = 100


In [12]:
optimizers = [ 'Adam'
        
    
]
for optimizer in optimizers:
    print(optimizer)
    modelnew = Model(inputs=model.input, outputs=FC_Head)
    modelnew.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  # tensorboard = TensorBoard(os.path.join('log_test4',f'{optimizer}_{time.time()}'))
    classifier = modelnew.fit_generator(
        train_generator,
        steps_per_epoch = nb_train_samples,
        epochs = epochs,
        callbacks = callbacks,
        validation_data = validation_generator,
        validation_steps = nb_validation_samples)
    modelnew.save("vgg16-transfer-learning-1.h5")
    


Adam
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
43/43 [==============================] - ETA: 0s - loss: 2.2265 - accuracy: 0.3637
Epoch 00001: val_loss improved from inf to 1.01845, saving model to vgg16-transfer-learning-1.h5
43/43 [==============================] - 31s 712ms/step - loss: 2.2265 - accuracy: 0.3637 - val_loss: 1.0185 - val_accuracy: 0.5990
Epoch 2/100
43/43 [==============================] - ETA: 0s - loss: 1.0738 - accuracy: 0.5636
Epoch 00002: val_loss improved from 1.01845 to 0.87903, saving model to vgg16-transfer-learning-1.h5
43/43 [==============================] - 28s 658ms/step - loss: 1.0738 - accuracy: 0.5636 - val_loss: 0.8790 - val_accuracy: 0.6510
Epoch 3/100
43/43 [==============================] - ETA: 0s - loss: 0.9631 - accuracy: 0.6082
Epoch 00003: val_loss improved from 0.87903 to 0.81009, saving model to vgg16-transfer-learning-1.h5
43/43 [==============================] - 29s 670ms/step - loss: 0.9631 

In [17]:
#loading saved model for prediction

from keras.models import load_model
classifier = load_model('vgg16-transfer-learning-1.h5')

In [18]:
import os
import cv2
import numpy as np
from os import listdir

from os.path import isfile, join

material_tags = {"[0]": "carton", 
                      "[1]": "vidrio",
                      "[2]": "metal",
                      "[3]": "papel",
                      "[4]": "plastico"
                      }


def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + str(path_class))
    file_path = path + "/" + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    


input_im = getRandomImage(test_data_dir)
input_original = input_im.copy()
input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
input_im = input_im / 255.
input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction

res = classifier.predict(input_im, verbose = 1)
print(res)

Class - metal
1/1 [==============================] - 0s 2ms/step
[[0.002984   0.15721013 0.7867592  0.04323382 0.00981286]]
